In [1]:
# ===================================================
# 🗄️ Notebook 7 — PostgreSQL Storage & Validation
# ===================================================
# Purpose:
#   Move processed data and SHAP results from CSVs into PostgreSQL tables.
#   Validate that data can be queried and reused by later apps.
#
# Inputs:
#   - data/processed/runs_summary.csv
#   - data/processed/shap_summary.csv
#   - src/db_utils.py  (for read/write helpers)
#
# Outputs:
#   - Tables in DB: runs_summary, shap_importance_global
#   - Validation queries confirming record counts & schema
# ===================================================

# ---------------------------------------------------
# 0️⃣  Project Path Setup
# ---------------------------------------------------
import sys
from pathlib import Path

# Detect project root (parent of the "notebooks" folder)
project_root = Path.cwd().parent
data_dir = project_root / "data" / "processed"

# Add src/ folder to Python path so imports work
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.db_utils import write_table, read_table, engine

print("✅ Project root:", project_root)
print("📂 Data dir:", data_dir)

# ---------------------------------------------------
# 1️⃣  Load Data
# ---------------------------------------------------
import pandas as pd

summary_path = data_dir / "runs_summary.csv"
shap_path = data_dir / "shap_summary.csv"

# Optional: check that files exist before reading
for path in [summary_path, shap_path]:
    if not path.exists():
        raise FileNotFoundError(f"❌ File not found: {path}")

# Load processed CSVs
summary_df = pd.read_csv(summary_path, parse_dates=["date"])
shap_summary = pd.read_csv(shap_path)

print("✅ Loaded data:")
print(f"  Runs summary: {summary_df.shape}")
print(f"  SHAP summary: {shap_summary.shape}")


✅ Project root: C:\Users\ITSMARTSOLUTIONS\documents\Python Scripts\running-agent
📂 Data dir: C:\Users\ITSMARTSOLUTIONS\documents\Python Scripts\running-agent\data\processed
✅ Loaded data:
  Runs summary: (826, 17)
  SHAP summary: (12, 1)


In [2]:
# ---------------------------------------------------
# 2️⃣  Write DataFrames → PostgreSQL
# ---------------------------------------------------
# Each write replaces the table if it already exists.
# You can switch mode="append" for incremental updates later.

write_table(summary_df, "runs_summary")
write_table(shap_summary, "shap_importance_global")


✅ Wrote 826 rows to table: runs_summary
✅ Wrote 12 rows to table: shap_importance_global


In [3]:
# ---------------------------------------------------
# 3️⃣  Quick Validation — record counts
# ---------------------------------------------------
import pandas as pd

print("📊 Row counts:")
for tbl in ["runs_summary", "shap_importance_global"]:
    count = pd.read_sql(f"SELECT COUNT(*) FROM {tbl};", engine)
    print(f"  {tbl:<25} → {count.iloc[0,0]} rows")


📊 Row counts:
  runs_summary              → 826 rows
  shap_importance_global    → 12 rows


In [4]:
# ---------------------------------------------------
# 4️⃣  Schema Inspection
# ---------------------------------------------------
# Check column names and data types stored in PostgreSQL

schema_info = pd.read_sql("""
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema='public'
    ORDER BY table_name, ordinal_position;
""", engine)

display(schema_info)


,table_name,column_name,data_type
0,runs_summary,run_id,bigint
1,runs_summary,date,timestamp without time zone
2,runs_summary,total_distance_km,double precision
3,runs_summary,duration_min,double precision
4,runs_summary,avg_pace_min_km,double precision
5,runs_summary,min_pace_min_km,double precision
6,runs_summary,fastest_1km_pace,double precision
7,runs_summary,fastest_5min_pace,double precision
8,runs_summary,avg_cadence,double precision
9,runs_summary,total_elev_gain,double precision


In [10]:
pd.read_sql("""
SELECT column_name 
FROM information_schema.columns 
WHERE table_name='shap_importance_global';
""", engine)

,column_name
0,0


In [11]:
# ---------------------------------------------------
# 5️⃣  Sample Query Checks
# ---------------------------------------------------
# Demonstrate analytical access through SQL queries
# Adapted for PostgreSQL (explicit numeric casts)
# ---------------------------------------------------

# --- Weekly summary (aggregated training stats) ---
query_1 = """
SELECT week,
       COUNT(*) AS n_runs,
       ROUND(AVG(total_distance_km)::numeric, 2) AS avg_distance,
       ROUND(AVG(avg_pace_min_km)::numeric, 2) AS avg_pace
FROM runs_summary
GROUP BY week
ORDER BY week;
"""
display(pd.read_sql(query_1, engine))

# --- SHAP importance check (top 10 features) ---
query_2 = """
SELECT "0" AS mean_importance
FROM shap_importance_global
ORDER BY "0" DESC
LIMIT 10;
"""
display(pd.read_sql(query_2, engine))



,week,n_runs,avg_distance,avg_pace
0,2014-09-29,12,4.77,NaN
1,2014-10-06,3,2.49,NaN
2,2014-10-13,2,13.07,NaN
3,2014-10-20,3,8.31,NaN
4,2014-10-27,3,9.02,NaN
...,...,...,...,...
204,2025-08-11,1,9.01,5.46
205,2025-08-25,1,11.72,6.22
206,2025-09-01,1,4.98,5.15
207,2025-09-08,2,10.79,12.56


,mean_importance
0,0.165745
1,0.150852
2,0.082938
3,0.080558
4,0.040268
5,0.030524
6,0.024739
7,0.023155
8,0.016385
9,0.008367


In [12]:
# ---------------------------------------------------
# 6️⃣  Data Lineage / Metadata (optional)
# ---------------------------------------------------
# Example: log dataset versions and timestamps for reproducibility.

import datetime, getpass
meta = pd.DataFrame([{
    "dataset": "runs_summary",
    "records": len(summary_df),
    "updated_at": datetime.datetime.now(),
    "user": getpass.getuser()
}])
write_table(meta, "data_lineage", mode="append")

print("🧾 Logged update to 'data_lineage' table.")


✅ Wrote 1 rows to table: data_lineage
🧾 Logged update to 'data_lineage' table.


In [13]:
# ---------------------------------------------------
# ✅ Notebook 7 — Wrap-Up & Validation Summary
# ---------------------------------------------------
# PostgreSQL integration is working end-to-end:
# - Runs and SHAP summaries successfully loaded from CSVs
# - Data written into the running_agent database
# - Analytical queries validated (weekly stats + feature importances)
# ---------------------------------------------------

print("🎉 PostgreSQL storage pipeline validated!\n")

print("📊 Tables now available in your database:")
for tbl in ["runs_summary", "shap_importance_global", "data_lineage"]:
    try:
        count = pd.read_sql(f"SELECT COUNT(*) FROM {tbl};", engine).iloc[0, 0]
        print(f"   • {tbl:<25} → {count} rows")
    except Exception:
        print(f"   • {tbl:<25} → (not found)")

print("\n🔗 Next steps:")
print("  1️⃣ Add cluster labels to runs_summary (via Notebook 3).")
print("  2️⃣ Re-export shap_summary with proper feature names.")
print("  3️⃣ Extend with interactive analytics in Notebook 8 (SQL + Streamlit).")



🎉 PostgreSQL storage pipeline validated!

📊 Tables now available in your database:
   • runs_summary              → 826 rows
   • shap_importance_global    → 12 rows
   • data_lineage              → 1 rows

🔗 Next steps:
  1️⃣ Add cluster labels to runs_summary (via Notebook 3).
  2️⃣ Re-export shap_summary with proper feature names.
  3️⃣ Extend with interactive analytics in Notebook 8 (SQL + Streamlit).
